<a href="https://colab.research.google.com/github/BokyungQQ/davethediver_analysis/blob/main/dave%20the%20diver%20%EC%BB%A4%EB%AE%A4%EB%8B%88%ED%8B%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 커뮤니티 키워드 분석

In [2]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 45.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt # 형태소 분석기 Okt
from collections import Counter

In [4]:
from konlpy.tag import Komoran

In [5]:
community = pd.read_csv('/content/데이브더다이버갤러리크롤링.csv')
community

,글번호,제목,댓글 개수,작성일자,조회수,추천
0,429,\n28일 23시59분도 28일임 ㅋㅋ,[1],23.06.28,212,1
1,428,\n야간손님 24명,[1],23.06.28,375,1
2,427,\n라이트닝 무기에 관한 모든것,[2],23.06.28,3306,13
3,426,\n정식 출시까지 20시간 전,[1],23.06.28,321,2
4,425,\n디데이 ㄷㄷㄷ,NaN,23.06.28,80,1
...,...,...,...,...,...,...
395,9,\n진짜 뭔 일 난건가 싶더라,[5],23.04.28,605,2
396,8,\n전주딱 미친새끼냐?,NaN,23.04.28,260,3
397,7,\n이유도 없이 그냥 멀쩡히 굴러가던 갤 폭파시킨건,[1],23.04.27,368,1
398,6,\n부활한 갤의 6번째 글은 내꺼다,NaN,23.04.27,69,0


In [6]:
# 정규 버전 발매일인 23년 6월 28일 게시글은 제외
community = community[community['작성일자']!='23.06.28']
community

,글번호,제목,댓글 개수,작성일자,조회수,추천
5,424,\n넥슨 대기업 아니었냐,[2],23.06.27,269,0
6,423,\n오랜만에 하는데 상어 너프된거같노,NaN,23.06.27,219,0
7,422,\n내일 저녁 오픈,[15],23.06.27,3135,13
8,421,\n30분뒤 열리네 ㅇㅇ,NaN,23.06.27,158,0
9,420,\n텄다 자자,NaN,23.06.27,69,1
...,...,...,...,...,...,...
395,9,\n진짜 뭔 일 난건가 싶더라,[5],23.04.28,605,2
396,8,\n전주딱 미친새끼냐?,NaN,23.04.28,260,3
397,7,\n이유도 없이 그냥 멀쩡히 굴러가던 갤 폭파시킨건,[1],23.04.27,368,1
398,6,\n부활한 갤의 6번째 글은 내꺼다,NaN,23.04.27,69,0


In [ ]:
# 명사만 추출
tokenizer_noun = Okt()
community['tokenized noun'] = community['제목'].apply(tokenizer_noun.nouns)

<ipython-input-29-b14de4f8bdc4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  community['tokenized noun'] = community['제목'].apply(tokenizer_noun.nouns)


In [ ]:
# 명사가 아니라 형태소 형태로 존재하는 단어들도 확인하기 위해 형태소 추출
tokenizer = Okt()
community['tokenized phrases'] = community['제목'].apply(tokenizer.morphs)
community['tokenized phrases']

<ipython-input-32-0e672467c980>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  community['tokenized phrases'] = community['제목'].apply(tokenizer.morphs)


5                             [\n, 넥슨, 대기업, 아니었냐]
6             [\n, 오랜, 만, 에, 하는데, 상어, 너프, 된거, 같노]
7                                [\n, 내일, 저녁, 오픈]
8                           [\n, 30분, 뒤, 열리네, ㅇㅇ]
9                                    [\n, 텄다, 자자]
                          ...                    
395                    [\n, 진짜, 뭔, 일, 난건, 가, 싶더라]
396                   [\n, 전, 주, 딱, 미친, 새끼, 냐, ?]
397    [\n, 이유, 도, 없이, 그냥, 멀쩡히, 굴러가던, 갤, 폭파, 시킨건]
398             [\n, 부활, 한, 갤의, 6, 번째, 글, 은, 내꺼다]
399                                      [\n, 부활]
Name: tokenized phrases, Length: 395, dtype: object

In [ ]:
# 불용어 설정
stop_words = ['진짜','아주','정도','이것','저것','가성','완전','정말','1','2','3','4','5','6','이거','지금','이','\n','게',"?","에","가","는","들","도","거","대","임","안","은","??","한","-","다","함","m","넥슨","시간",
              '겜', '더', '제발', '것', '수', '때', '좀','왜','그', '내','의','를','로','을','너무','게임']

In [ ]:
community

,글번호,제목,댓글 개수,작성일자,조회수,추천,tokenized noun,tokenized phrases
5,424,\n넥슨 대기업 아니었냐,[2],23.06.27,269,0,"[넥슨, 대기업]","[\n, 넥슨, 대기업, 아니었냐]"
6,423,\n오랜만에 하는데 상어 너프된거같노,NaN,23.06.27,219,0,"[만, 상어, 너프, 된거]","[\n, 오랜, 만, 에, 하는데, 상어, 너프, 된거, 같노]"
7,422,\n내일 저녁 오픈,[15],23.06.27,3135,13,"[내일, 저녁, 오픈]","[\n, 내일, 저녁, 오픈]"
8,421,\n30분뒤 열리네 ㅇㅇ,NaN,23.06.27,158,0,[],"[\n, 30분, 뒤, 열리네, ㅇㅇ]"
9,420,\n텄다 자자,NaN,23.06.27,69,1,[텄다],"[\n, 텄다, 자자]"
...,...,...,...,...,...,...,...,...
395,9,\n진짜 뭔 일 난건가 싶더라,[5],23.04.28,605,2,"[진짜, 일, 난건]","[\n, 진짜, 뭔, 일, 난건, 가, 싶더라]"
396,8,\n전주딱 미친새끼냐?,NaN,23.04.28,260,3,[새끼],"[\n, 전, 주, 딱, 미친, 새끼, 냐, ?]"
397,7,\n이유도 없이 그냥 멀쩡히 굴러가던 갤 폭파시킨건,[1],23.04.27,368,1,"[이유, 그냥, 폭파]","[\n, 이유, 도, 없이, 그냥, 멀쩡히, 굴러가던, 갤, 폭파, 시킨건]"
398,6,\n부활한 갤의 6번째 글은 내꺼다,NaN,23.04.27,69,0,"[부활, 글]","[\n, 부활, 한, 갤의, 6, 번째, 글, 은, 내꺼다]"


In [ ]:
# ['tokenized'].values에서 단어 추출
words = np.hstack(community['tokenized noun'].values)
filtered_words = [word for word in words]

# 단어의 빈도수를 계산
word_count = Counter(filtered_words)
word_count = Counter({x : word_count[x] for x in word_count \
                               if x not in stop_words})
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
for word, count in word_count:
    print(f"{word}: {count}번")

출시: 46번
정식: 33번
맵: 15번
언제: 15번
뭐: 13번
오픈: 12번
초밥: 9번
개: 9번
물고기: 9번
노가다: 9번
정리: 8번
얼: 8번
데이브: 8번
뭐임: 7번
생각: 7번
점: 7번
추가: 7번
백미: 7번
무기: 7번
내일: 6번
스팀: 6번
전: 6번
다시: 6번
직원: 6번
영상: 6번
연안: 6번
업글: 6번
양식: 6번
돈: 6번
정보: 6번
엑세스: 5번
정발: 5번
이제: 5번
기대: 5번
리뷰: 5번
나: 5번
시작: 5번
재: 5번
다이버: 5번
컨텐츠: 5번
재료: 5번
씨발: 5번
버전: 5번
난: 4번
사람: 4번
가격: 4번
수면: 4번
스포: 4번
질문: 4번
오늘: 4번
요리: 4번
메뉴: 4번
데더: 4번
해초: 4번
주: 4번
농장: 4번
정: 4번
시발: 4번
어디: 4번
글: 4번
트레일러: 4번
출시일: 4번
부활: 4번
상어: 3번
특: 3번
어케: 3번
한국: 3번
햇: 3번
차이: 3번
듯: 3번
옴: 3번
이유: 3번
발매: 3번
마지막: 3번
이번: 3번
일: 3번
식당: 3번
총: 3번
가장: 3번
작년: 3번
사서: 3번
잠입: 3번
재미: 3번
아이템: 3번
또: 3번
반: 3번
처음: 3번
스시: 3번
보스: 3번
이틀: 3번
종유: 3번
동굴: 3번
혹시: 3번
화염: 3번
스나이퍼: 3번
그냥: 3번
달: 3번
부러: 3번
이빨: 3번
인터뷰: 3번
업로드: 3번
하나: 3번
농사: 3번
대체: 3번
소식: 3번
된거: 2번
밤: 2번
작살: 2번
후: 2번
몇: 2번
년: 2번
공식: 2번
말: 2번
스위치: 2번
스토리: 2번
진행: 2번
그대로: 2번
장사: 2번
준비: 2번
머: 2번
해마: 2번
종류: 2번
업데이트: 2번
벌써: 2번
장면: 2번
건가: 2번
해: 2번
고유: 2번
방식: 2번
그래비티: 2번
가오리: 2번
수확: 2번
노잼: 2번
안나: 2번
중: 2번
액션: 2번
해도: 2번
미니게임: 2번
명: 2번
점수: 2번
버튼: 2번
크기: 2번
못: 2번


In [ ]:
# 단어 기준으로 키워드 빈도를 확인하여 대략적인 빈도 확인

# ['tokenized'].values에서 단어 추출
words = np.hstack(community['tokenized
'].values)
filtered_words = [word for word in words]

# 단어의 빈도수를 계산
word_count = Counter(filtered_words)
word_count = Counter({x : word_count[x] for x in word_count \
                               if x not in stop_words})
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
for word, count in word_count:
    print(f"{word}: {count}번")

출시: 46번
정식: 33번
6월: 22번
맵: 15번
언제: 15번
D: 15번
뭐: 13번
오픈: 12번
까지: 12번
나옴: 12번
하면: 12번
으로: 11번
): 11번
개: 10번
초밥: 9번
갤: 9번
아: 9번
물고기: 9번
노가다: 9번
정리: 8번
28일: 8번
몇: 8번
인데: 8번
하: 8번
얼: 8번
엑: 8번
데이브: 8번
있음: 8번
(: 8번
뭐임: 7번
전: 7번
얼리: 7번
생각: 7번
하고: 7번
점: 7번
추가: 7번
.: 7번
백미: 7번
면: 7번
부터: 7번
주: 7번
무기: 7번
장: 7번
내일: 6번
스팀: 6번
다시: 6번
못: 6번
냐: 6번
할: 6번
나: 6번
직원: 6번
영상: 6번
part: 6번
연안: 6번
0-50: 6번
업글: 6번
좋겠다: 6번
양식: 6번
돈: 6번
정보: 6번
만: 5번
???: 5번
하는거: 5번
엑세스: 5번
정발: 5번
이제: 5번
총: 5번
기대: 5번
리뷰: 5번
고: 5번
..: 5번
시작: 5번
...: 5번
재: 5번
다이버: 5번
컨텐츠: 5번
,: 5번
재료: 5번
어: 5번
씨발: 5번
버전: 5번
시: 4번
해야: 4번
난: 4번
일: 4번
이야: 4번
사람: 4번
가격: 4번
지: 4번
수면: 4번
이다: 4번
스포: 4번
질문: 4번
오늘: 4번
해: 4번
야: 4번
ㅋㅋ: 4번
요리: 4번
중: 4번
해도: 4번
어인: 4번
메뉴: 4번
인가: 4번
데더: 4번
해초: 4번
에서: 4번
됨: 4번
빨리: 4번
농장: 4번
나오면: 4번
정: 4번
적: 4번
인: 4번
시발: 4번
딱: 4번
되는거: 4번
쓰: 4번
어디: 4번
글: 4번
트레일러: 4번
되면: 4번
출시일: 4번
부활: 4번
상어: 3번
특: 3번
기: 3번
어케: 3번
00시: 3번
걍: 3번
한국: 3번
햇: 3번
는데: 3번
랑: 3번
차이: 3번
ㅋㅋㅋ: 3번
듯: 3번
옴: 3번
이유: 3번
발매: 3번
마지막: 3번
!: 3번
~: 3번
새로: 3번
할까: 3번
이번: 3번


# 커뮤니티 게시글, 조회수 비중 확인 ( 정식 출시 / 맵 )

In [7]:
# 전체 게시글 개수 확인 및 조회수 확인
print('게시글 개수 (전체) : ',len(community))
print('게시글 전체 조회수 (전체) : ',sum(community['조회수']))

게시글 개수 (전체) :  395
게시글 전체 조회수 (전체) :  279297


In [8]:
# 정식 출시와 관련된 게시글만 추출
real = [(title, views) for title, views in zip(community['제목'], community['조회수']) if '정식' in title or '출시' in title or '6월' in title or '언제' in title ]
real
# 데이터 프레임으로 변경
real_df = pd.DataFrame(real, columns=['제목', '조회수'])
real_df

,제목,조회수
0,\n내일 출시랑 스팀에 지금 있는거랑 차이가 뭐임??,205
1,"\n니미 몇시간 후에 출시하는줄,,,ㅋㅋㅋ",217
2,\n그럼 출시 사실상 29일임?,319
3,\n출시 날자 현지화 안 하는거 개에바인데,366
4,\n스위치로 언제 나옴?,312
...,...,...
77,\n빨리 정식출시 하면 좋겠다.,167
78,\n정식출시 하면 엄청 크게 달라질 예정임?,562
79,\n정식 출시 언제대냐,162
80,\n그래서 정식출시 언제됨,337


In [9]:
# 정식 출시와 관련된 게시글 비중 및 조회수 확인
print('게시글 개수 (정식 출시) : ',len(real_df), " | 비중 : ",round(len(real_df)/len(community)*100,2), "%")
print('게시글 전체 조회수 (정식 출시) : ',sum(real_df['조회수'])," | 비중 : ",round(sum(real_df['조회수'])/sum(community['조회수'])*100,2), "%")
print("게시글 작성 대비 조회수 비율 (정식 출시) :", round(sum(real_df['조회수'])/ len(real_df),2),"배" )

게시글 개수 (정식 출시) :  82  | 비중 :  20.76 %
게시글 전체 조회수 (정식 출시) :  28108  | 비중 :  10.06 %
게시글 작성 대비 조회수 비율 (정식 출시) : 342.78 배


In [10]:
# 맵과 지도가 들어간 게시글 비중 및 조회수 확인
map = [(title, views) for title, views in zip(community['제목'], community['조회수']) if '맵' in title or '지도' in title]
map
# 데이터 프레임으로 변경
map_df = pd.DataFrame(map, columns=['제목', '조회수'])
map_df

,제목,조회수
0,\n밤 맵 정리,6821
1,\n맵 정리 part 연안6 0-50m,4970
2,\n50m 6번째 맵이 있었음...,147
3,\n이제 원하는 맵 가챠하면 입구 수에 따라 구별하자,117
4,\n혹시 비오는 날/맑은 날 맵이 다른가?,224
5,\n50-130m 맵 다 그렸는데 의견이 필요함,251
6,\n50-130m 맵 몇개일까?,123
7,"\n50-130m 맵(완, 수정9차)",16923
8,\n맵 정리 part 연안1 0-50m,12603
9,\n50m 맵 마지막 다끝나간다... / 업뎃 분량 예상해보기,444


In [11]:
# 맵에 관련된 게시글 비중 및 조회수 확인
print('게시글 개수 (맵) : ',len(map_df), " | 비중 : ",round(len(map_df)/len(community)*100,2), "%")
print('게시글 전체 조회수 (맵) : ',sum(map_df['조회수'])," | 비중 : ",round(sum(map_df['조회수'])/sum(community['조회수'])*100,2), "%")
print("게시글 작성 대비 조회수 비율:", round(sum(map_df['조회수'])/len(map_df),2),"배" )

게시글 개수 (맵) :  17  | 비중 :  4.3 %
게시글 전체 조회수 (맵) :  103017  | 비중 :  36.88 %
게시글 작성 대비 조회수 비율: 6059.82 배


## 맵에 관련된 게시글 세부 의견 확인

In [13]:
# 맵 : 게시글 개수 비중 확인
print("맵이 존재하지 않는 것에 대한 불만 : ", 1, " | 비중 : ", round((1/17) * 100,2),"%")
print("맵에 대한 궁금증 : ", 4, " | 비중 : ", round((4/17) * 100,2),"%")
print("정리( 맵을 정리한 게시글 ) : ", 12," | 비중 : ", round((12/17) * 100,2),"%")

맵이 존재하지 않는 것에 대한 불만 :  1  | 비중 :  5.88 %
맵에 대한 궁금증 :  4  | 비중 :  23.53 %
정리( 맵을 정리한 게시글 ) :  12  | 비중 :  70.59 %


In [12]:
# 맵 : 게시글 조회수 비중 확인
# 요소들의 위치를 원하는 게시글 : 1개
print("맵이 존재하지 않는 것에 대한 불만 : ", 177, " | 비중 : ", round((177/sum(map_df['조회수'])) * 100,2),"%")
# 맵에 대한 궁금증 : 4개
map4 = 147+224+251+123
print("맵에 대한 궁금증 : ", map4 , " | 비중 : ", round((map4/sum(map_df['조회수'])) * 100,2),"%")
# 정리( 맵을 정리한 게시글 ) : 12개
map12 = sum(map_df['조회수'] - (177+map4))
print("정리( 맵을 정리한 게시글 ) : ", map12," | 비중 : ", round((map12/sum(map_df['조회수'])) * 100,2),"%")

맵이 존재하지 않는 것에 대한 불만 :  177  | 비중 :  0.17 %
맵에 대한 궁금증 :  745  | 비중 :  0.72 %
정리( 맵을 정리한 게시글 ) :  87343  | 비중 :  84.79 %


# 전체 조회수 상위 5개 게시글 확인

In [ ]:
# 전체 조회수 상위 5개 게시글
top_5 = community.sort_values(by='조회수',ascending=False).head(5)
top_5

# 정리 게시글 : 3
# 공략 게시글 : 1
# 정리된 내용을 원하는 게시글 : 1

,글번호,제목,댓글 개수,작성일자,조회수,추천,tokenized noun,tokenized phrases
302,108,\n지도,[6],23.06.08,33970,43,[지도],"[\n, 지도]"
205,208,\n해마 공략,[9],23.06.17,17647,33,"[해마, 공략]","[\n, 해마, 공략]"
135,284,"\n50-130m 맵(완, 수정9차)",[16],23.06.23,16923,47,"[맵, 완, 수정, 차]","[\n, 50-130, m, 맵, (, 완, ,, 수정, 9, 차, )]"
143,276,\n맵 정리 part 연안1 0-50m,[9],23.06.22,12603,17,"[맵, 정리, 연안]","[\n, 맵, 정리, part, 연안, 1, 0-50, m]"
301,109,\n종유 동굴 랜덤으로 출현함?,[4],23.06.08,12237,2,"[종유, 동굴, 랜덤, 출현, 함]","[\n, 종유, 동굴, 랜덤, 으로, 출현, 함, ?]"
